In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv('data/Telco-Customer-Churn.train.csv')



### Visualizzo il dataset

In [ ]:
dataset

### Vedo le chiavi presenti nel database

In [ ]:
for el in dataset.columns:
    print("Colonna:"+el+"",dataset[el].unique())

In [ ]:
dataset.TotalCharges.head(20)

In [ ]:
dataset= dataset.replace(' ', np.NaN)

### Vado a buttare le poche righe che presentano dei valori nulli nella colonna "TotalCharges"

In [ ]:
dataset = dataset.dropna()
dataset.info()

### Per poter lavorare in comodità vado a separare i valori categorici da quelli numerico

In [ ]:
import numpy as np

In [ ]:
features_numeriche = dataset.select_dtypes(include = np.number)
features_numeriche

In [ ]:
features_categoriche = dataset.select_dtypes(include = 'object')
features_categoriche

## Label Encoding

Vado ad utilizzare il label encoder per trasformare i dati categorici del dataset(di tipo 'object') in valori interi.


In [ ]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()

for i in features_categoriche:
    features_categoriche[i] = label_encoder.fit_transform(features_categoriche[i])
    
features_categoriche

### Vado a ricompattare i dati che precedentemente avevo separato in numerici e categorici, nel dataset intero

In [ ]:
fullDatasetNumCat = pd.concat([features_numeriche, features_categoriche], axis = 1)

fullDatasetNumCat

### Separo il target dal resto dei dati

In [ ]:
y = fullDatasetNumCat.Churn

X = fullDatasetNumCat.drop(columns=['Churn'])

In [ ]:
y

In [ ]:
X

In [ ]:
X

## Features Selection

Per andare a semplicare il modello ed evitare un eccessivo overfitting vado ad utilizzare, per la scelta di un sottoinsieme dei vari attributi, **SelectFromModel**.

Questo strumento, offerto da sklearn, è di fatto un meta-trasformatore che a seconda dello stimatore che gli viene passato, va a fare una selezione delle features ad-hoc.
In questo caso ho scelto di utilizzare la **RidgeRegression**, che è un ottimo strumento per poter andare a diminuire l'overfitting.


L’idea alla base della Ridge Regression è quella di limitare il valore assoluto dei coefficienti **Wi** definendo come segue la funzione di costo totale (da minimizzare nella fase di training):

>> $$costoRidge = misura del “fit” + misura della grandezza dei coefficienti$$

Con misura del fit, che rappresenta una qualsiasi funzione di costo come per esempio l'**RSS**, mentre la grandezza dei coefficienti, per quanto riguarda la RidgeRegression, è rappresentata dall'utilizzo della **l2 norm**.

La **l2 norm** è il fattore di regolarizzazione rappresentato dalla seguente espressione:
>> $$R(W) = \sum _l (w_l)^2$$

Questo parametro di regolarizzazione va a mettere in quadratura tutti i parametri.Successivamente li vado a sommare.
Con parametri piccoli la quadratura mi fa scendere il costo(perchè se ho valori <1, andando a fare il quadrato questi mi diventano ancora più piccoli), con parametri sopra l'1 andrà ad aumentare(perchè ovviamente il quadrato sarà un valore più grande).

La Ridge può quindi essere riscritta come:
>> $$costoRidge(w) = RSS(w) + \lambda * R(w)$$

Come si può notare, affianco alla nostra **l2 norm**, compare un parametro **lambda**. Questo parametro viene detto di tuning, e sostanzialmente serve a bilanciare quella che è l'espressione sopra descritta. **Lambda** può assumere sostanzialmente vari valori e a seconda di quelli che assume abbiamo che la nostra funzione di costo adotta un comportamento differente. Infatti:

1) se lambda=0, come si può facilmente osservare abbiamo che la nostra funzione di costo ridge diventa sostanzialmente una RSS.

2) se lambda invece tende ad infinito, allora abbiamo che l'intero costo mi tende ad infinito e l'unico modo per minimizzare è quello di avere W=0.

Generalmente per alti valori di lambda abbiamo inoltre un alto bias e una bassa vairianza, mentre per bassi valori il viceversa.

In [ ]:
from sklearn.feature_selection import SelectFromModel
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso,Ridge




#sfm=SelectFromModel(LogisticRegression(max_iter=100000, C=1000))
sfm=SelectFromModel(Ridge(max_iter=100000, alpha= 0.001))
sfm.fit(X, y)

sfm.get_support()

selected_Ridge = X.columns[(sfm.get_support())]

selected_Ridge

In [ ]:
X = X[selected_Ridge]
X

## ADDESTRAMENTO DEL MODELLO

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


tr = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=6)


#fit e prediction
tr.fit(X_train, y_train)
pred = tr.predict(X_valid)
print('Accuracy:',accuracy_score(y_valid, pred))